## Simulação usinagem

Nesta simulação, vamos avaliar o processo de usinagem em uma célula de montagem. Para tanto, vamos inicialmente definir os parâmetros que serão utilizados na simulação e os processos que serão realizados.

In [1]:
import simpy
import random
import numpy as np

TEMPO_SIMULACAO = 8 * 60
MEDIA_CHEGADA_PALLETS = 40
DESVIO_PADRAO_CHEGADA_PALLETS = 3
MEDIA_FIXACAO = 40 / 60
DESVIO_PADRAO_FIXACAO = 5 / 60
TEMPO_USINAGEM = 3
TEMPO_PARAFUSAGEM_MIN = 3.5
TEMPO_PARAFUSAGEM_MAX = 4

estatisticas = {
    'tempo_espera_fixacao': [],
    'tempo_espera_usinagem': [],
    'tempo_espera_parafusagem': [],
    'tempo_total_processamento': [],
    'total_pecas_processadas': 0,
    'tempo_ocio_funcionario1': 0,
    'tempo_ocio_funcionario2': 0
}


def tempo_chegada_pallets():
    """Gera o tempo de chegada dos pallets, seguindo uma distribuição normal."""
    return random.normalvariate(MEDIA_CHEGADA_PALLETS,
                                DESVIO_PADRAO_CHEGADA_PALLETS)


def tempo_fixacao():
    """Gera o tempo de fixação das peças, seguindo uma distribuição normal."""
    return max(0, random.normalvariate(MEDIA_FIXACAO, DESVIO_PADRAO_FIXACAO))


def tempo_parafusagem():
    """Gera o tempo de parafusagem, seguindo uma distribuição uniforme."""
    return random.uniform(TEMPO_PARAFUSAGEM_MIN, TEMPO_PARAFUSAGEM_MAX)


def processo_par_pecas(env, fixacao, usinagem, parafusagem):
    """Simula o processamento de um par de peças, passando pelas etapas de fixação, usinagem e parafusagem."""
    inicio_processo = env.now

    yield from realizar_etapa(env, fixacao, tempo_fixacao, 'fixacao')
    yield from realizar_etapa(env, usinagem, lambda: TEMPO_USINAGEM,
                              'usinagem')
    yield from realizar_etapa(env, parafusagem, tempo_parafusagem,
                              'parafusagem')
    tempo_total = env.now - inicio_processo
    estatisticas['tempo_total_processamento'].append(tempo_total)


def realizar_etapa(env, recurso, tempo_operacao, nome_etapa):
    """Realiza uma etapa do processo (fixação, usinagem ou parafusagem), medindo o tempo de espera e operação."""
    inicio_espera = env.now
    with recurso.request() as req:
        yield req
        tempo_espera = env.now - inicio_espera
        estatisticas[f'tempo_espera_{nome_etapa}'].append(tempo_espera)
        yield env.timeout(tempo_operacao())


def chegada_pallets(env, fixacao, usinagem, parafusagem):
    """Simula a chegada de pallets e inicia o processamento dos pares de peças."""
    while True:
        yield env.timeout(tempo_chegada_pallets())
        estatisticas['total_pecas_processadas'] += 60
        for _ in range(30):
            env.process(processo_par_pecas(env, fixacao, usinagem,
                                           parafusagem))


def monitor_ociosidade(env, fixacao, usinagem, parafusagem):
    """Monitora o tempo ocioso dos funcionários durante a simulação."""
    while True:
        if fixacao.count == 0 and usinagem.count == 0:
            estatisticas['tempo_ocio_funcionario1'] += 1
        if parafusagem.count == 0:
            estatisticas['tempo_ocio_funcionario2'] += 1
        yield env.timeout(1)


def simular(proxima_decisao):
    """Executa a simulação com ou sem a decisão adicional de permitir que o operador da usinagem faça a parafusagem."""
    env = simpy.Environment()

    fixacao = simpy.Resource(env, capacity=1)
    usinagem = simpy.Resource(env, capacity=1)
    parafusagem_capacity = 2 if proxima_decisao else 1
    parafusagem = simpy.Resource(env, capacity=parafusagem_capacity)

    env.process(chegada_pallets(env, fixacao, usinagem, parafusagem))
    env.process(monitor_ociosidade(env, fixacao, usinagem, parafusagem))

    env.run(until=TEMPO_SIMULACAO)

    mostrar_resultados()


def mostrar_resultados():
    """Exibe as estatísticas geradas durante a simulação."""
    print(
        f"Tempo médio de espera na fixação: {np.mean(estatisticas['tempo_espera_fixacao']):.2f} minutos"
    )
    print(
        f"Tempo médio de espera na usinagem: {np.mean(estatisticas['tempo_espera_usinagem']):.2f} minutos"
    )
    print(
        f"Tempo médio de espera na parafusagem: {np.mean(estatisticas['tempo_espera_parafusagem']):.2f} minutos"
    )
    print(
        f"Tempo médio total de processamento: {np.mean(estatisticas['tempo_total_processamento']):.2f} minutos"
    )
    print(
        f"Número total de peças processadas: {estatisticas['total_pecas_processadas']}"
    )
    print(
        f"Tempo ocioso do funcionário 1: {estatisticas['tempo_ocio_funcionario1']} minutos"
    )
    print(
        f"Tempo ocioso do funcionário 2: {estatisticas['tempo_ocio_funcionario2']} minutos"
    )


def reiniciar_estatisticas():
    """Reinicia todas as estatísticas para uma nova simulação."""
    for chave in estatisticas:
        if isinstance(estatisticas[chave], list):
            estatisticas[chave].clear()
        else:
            estatisticas[chave] = 0

In [26]:
print("Simulação sem permitir que o operador da usinagem faça a parafusagem:")
simular(proxima_decisao=False)

reiniciar_estatisticas()

print("\nSimulação permitindo que o operador da usinagem faça a parafusagem:")
simular(proxima_decisao=True)


Simulação sem permitir que o operador da usinagem faça a parafusagem:
Tempo médio de espera na fixação: 9.42 minutos
Tempo médio de espera na usinagem: 130.64 minutos
Tempo médio de espera na parafusagem: 19.77 minutos
Tempo médio total de processamento: 154.20 minutos
Número total de peças processadas: 1440
Tempo ocioso do funcionário 1: 88 minutos
Tempo ocioso do funcionário 2: 95 minutos

Simulação permitindo que o operador da usinagem faça a parafusagem:
Tempo médio de espera na fixação: 9.72 minutos
Tempo médio de espera na usinagem: 127.42 minutos
Tempo médio de espera na parafusagem: 0.00 minutos
Tempo médio total de processamento: 142.57 minutos
Número total de peças processadas: 660
Tempo ocioso do funcionário 1: 41 minutos
Tempo ocioso do funcionário 2: 44 minutos


Com os dados acima, podemos analisar individualmente cada uma das situações para entender suas diferenças e os gargalos presentes em cada uma delas.


### Sem permitir que o operador da usinagem faça a parafusagem:
- **Gargalo na usinagem**: O tempo médio de espera na usinagem foi muito elevado (130,64 minutos), indicando que a máquina de usinagem é o principal gargalo do processo.
- **Tempo de espera nas filas**: Houve um tempo considerável de espera nas filas de fixação (9,42 minutos) e parafusagem (19,77 minutos).
- **Peças processadas**: Foram processadas 1.440 peças, um valor alto, mas com muita ineficiência devido aos gargalos.
- **Ociosidade dos funcionários**: O funcionário 1 (fixação e usinagem) e o funcionário 2 (parafusagem) tiveram tempos ociosos semelhantes, indicando que ambos os processos geraram períodos de espera.

### Permitindo que o operador da usinagem faça a parafusagem:
- **Redução do gargalo na parafusagem**: O tempo de espera na parafusagem foi eliminado (0 minutos), pois o operador da usinagem ajudou a aliviar a fila.
- **Menor tempo de processamento total**: O tempo médio total de processamento foi reduzido de 154,20 para 142,57 minutos, uma melhoria significativa.
- **Menor número de peças processadas**: Embora o tempo total de processamento tenha melhorado, o número de peças processadas caiu drasticamente (660 peças), sugerindo que o operador ficou sobrecarregado ao realizar duas funções.
- **Redução da ociosidade dos funcionários**: O tempo ocioso de ambos os funcionários diminuiu, especialmente do funcionário 1, que agora realiza tanto a usinagem quanto a parafusagem.

### Conclusão:
Permitir que o operador da usinagem faça a parafusagem eliminou o gargalo na fila de parafusagem, mas sobrecarregou o operador, resultando em menos peças processadas. Isso sugere que a estratégia de combinar funções pode não ser eficaz para aumentar a capacidade total do sistema.